# Lab 5 - Leveraging Tasks

In this notebook, we will explore new aspects that can be added to tasks to enhance their performance. These include memory, Pydantic outputs, and human inputs.

## What are we going to cover in this notebook?

We will cover the following topics:

1. Introducing humans in the loop
2. Pydantic outputs
3. Adding memory
4. Example Use Case

## Libraries

In this section, we will import the necessary libraries, as done in all the previous labs. We'll be using the same use case example as the second lab, the article writing problem. We'll introduce those improvements to that same problem.

In [2]:
from crewai import Agent, Task, Crew, Process, LLM
from IPython.display import Markdown

You can manually instantiate the LLM class to modify the [LLM parameters](https://docs.crewai.com/how-to/LLM-Connections/). The default Ollama port is 11434.

In [3]:
ollama = LLM(
    model='ollama/llama3.1',
    base_url='http://localhost:11434',
)

## Agents

As a reminder, we had three different agents. The **planning** agent can focus on researching the topic, gathering relevant information, and outlining the article structure. Meanwhile, the **writing** agent can concentrate on crafting the actual content based on the findings from the planning agent. Finally, the **editor** can review the article for grammar, punctuation, and style consistency.

### Planner Agent

In [4]:
planner = Agent(
    role='Content Planner',
    goal='Plan engaging and factually accurate blog article content on {topic}.',
    backstory=(
        "You're working on planning a comprehensive blog article "
        "about the topic: {topic}."
        "Your responsibilities include conducting research, "
        "gathering data and statistics related to the topic, "
        "and identifying potential sources for quotes or expert opinions."
        "Your work will serve as the basis for the Content Writer "
        "to write an article on this topic."
    ),
    allow_delegation=False,
    verbose=True,
    llm=ollama
)

### Writer Agent

In [5]:
writer = Agent(
    role='Content Writer',
    goal='Write an insightful and factually accurate opinion piece about the topic: {topic}',
    backstory=(
        "You're working on writing a new opinion piece "
        "about the topic: {topic}. You base your writing "
        "on the work of the Content Planner, who provides "
        "an outline and relevant context about the topic. "
        "Your responsibilities include conducting research "
        "on relevant sources and references, gathering "
        "quotes or expert opinions from relevant experts, "
        "and ensuring that the opinion piece is well-structured "
        "and easy to read for the general public."
    ),
    allow_delegation=False,
    verbose=True,
    llm=ollama
)

### Editor Agent

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with the writing style of the organization, "
         "with a focus on clarity and concision.",
    backstory=(
        "You are an editor who receives a blog post "
        "from the Content Writer. "
        "Your responsibilities include conducting "
        "a fact-check on the blog post, ensuring that "
        "the language is free from bias and jargon, "
        "and reviewing the content for accuracy and completeness."
    ),
    allow_delegation=False,
    verbose=True,
    llm=ollama
)

## Tasks

We also had three different tasks, one for each agent. In the following task we'll introduce the human in the loop. That is, human intervention is required to check what the agents are doing. In this case, the human will be shown the output of a given task and will be able to provide feedback on those results. The agent will then use the provided feedback to proceed with its task.

To introduce human input in a task, `human_input=True` must be added as a task parameter. Optionally, you can also introduce in the task description a comment about the involvement of a human in that task. As an example, in our planning task, we've introduced a fifth step in the description and the _human_input_ parameter.

**Note**: human input does not work in VS Code at the time of writing this notebook.

### Plan Task

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources.\n"
        "5. Check with a human if the content plan is good before "
            " finalizing it."
    ),
    expected_output=(
        "A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
    ),
    agent=planner,
    human_input=True
)

### Write Task

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice."
    ),
    expected_output=(
        "A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs. "
    ),
    agent=writer
)

### Pydantic Output

In the edit task, we'll introduce Pydantic outputs. [Pydantic](https://docs.pydantic.dev/latest/) is a Python data validation library that helps define how data should be structured. In our use case, we'll apply it to ensure the output of a task adheres to our predefined structure.

To create a new Pydantic schema, simply subclass `BaseModel` and define the fields of your data model. For our purposes, we'll format the schema with the variable name, its type, and an accurate description of the content the variable holds. An example is as follows:

```python
var_name: var_type = Field(..., description='Description of the variable.')
```

Let's create an `ArticleSchema` for our edit task. We'll require the output of this task to follow our predefined structure.

Our model will have three outputs:
- The **title**, represented as a string (`str`)
- A brief **summary** of the article, also a string (`str`)
- The article's **paragraphs**, as a list of strings (`List[str]`), where each item in the list represents a paragraph

We are also going to create a `get_schema` method. This method will return the description of the schema, which we'll use to provide an explanation of this structured output to the agent. You can reuse this method for all the schemas you need. Just copy it and paste it inside the BaseModel subclass.

```python
@classmethod
def get_schema(cls) -> str:
    schema = '\n'
    for field_name, field_instance in cls.__fields__.items():
        schema += f'{field_name}, described as: {field_instance.description}\n'
    return schema
```

In [9]:
from pydantic import BaseModel, Field
from typing import List

class ArticleSchema(BaseModel):
    """Output for the article edit task."""
    title: str = Field(..., description='Title of the article.')
    summary: str = Field(..., description='Brief summary of the article.')
    paragraphs: List[str] = Field(..., description='Paragraphs of the article.')

    @classmethod
    def get_schema(cls) -> str:
        schema = '\n'
        for field_name, field_instance in cls.__fields__.items():
            schema += f'{field_name}, described as: {field_instance.description}\n'
        return schema

### Edit Task

To add a Pydantic output to this task, you need to provide the schema in the `output_pydantic` parameter. This is a straightforward way to ensure that the output adheres to our predefined structure.

We're also going to provide the schema (by using the `get_schema` method) to the expected output. Check the last line of the `expected_output` description for more information on this step.

Moreover, we'll create a new file containing this structured output as a JSON file. To do so, simply add `output_file='filename.json'`. An alternative approach would be to use the `output_json` parameter instead of `output_pydantic`. Since Pydantic can also output a JSON file, we'll stick with it. **Note**: Only one output type can be set at a time.

In [10]:
edit = Task(
    description=(
        "Proofread the given blog post for "
        "grammatical errors and "
        "alignment with the brand's voice."
    ),
    expected_output=(
        "A well-written blog post ready for "
        "publication. It must have a title, a "
        "summary, and a list of paragraphs."
        f"The output should adhere to the following schema: {ArticleSchema.get_schema()}"
    ),
    agent=editor,
    output_pydantic=ArticleSchema,
    output_file='Article.json'
)

## Crew and Memory

We'll create a sequential crew execution, as we did on the second lab, but this time we'll be adding [memory](https://docs.crewai.com/core-concepts/Memory/). The memory system helps agents to remember, reason, and learn from past interactions. That's why it has several memory types: Short-term, long-term, entity and contextual.

To use memory on a crew, you simply need to add the parameter `memory=True`, and in our case, configure an embedder model (mxbai-embed-large) using ollama.

There's also the option to [reset the memory](https://docs.crewai.com/core-concepts/Memory/) by executing the command `crewai reset_memories [OPTIONAL_PARAMETERS]` in the terminal.

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    memory=True,
    embedder=dict(
      provider='ollama',
      config=dict(
        model='mxbai-embed-large',
      ),
    )
)

## Running the improved crew

Now we can finally run our improved crew. Once the planning task is completed, you'll se how the planner agent will ask for feedback. It can be provided by writing in the text box. Until this interaction is completed, the execution will be on halt.

## Running the Improved Crew

Now that we have our crew configuration with memory capabilities, we can proceed to run the improved crew. Once the planning task is completed, you will see how the planner agent requests feedback. This interaction is crucial in completing the planning task.

**Note**: Until the planning task and subsequent feedback interaction are complete, the execution will be on hold.

In [12]:
result = crew.kickoff(inputs={'topic': 'Artificial Intelligence'})

# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
5. Check with a human if the content plan is good before  finalizing it.


# Agent: Content Planner
## Final Answer: 
**Comprehensive Content Plan Document**

**I. Introduction**

* Title: "Unlocking the Potential of Artificial Intelligence: Trends, Players, and Future Prospects"
* Tagline: "Discover how AI is revolutionizing industries and transforming lives"

**II. Target Audience Analysis**

* Demographics:
	+ Professionals working in technology, healthcare, finance, and education
	+ Students interested in AI and data science
	+ Individuals curious about the impact of AI on society
* Interests:
	+ Staying updated on the latest AI 

 The content plan seems good to me




# Agent: Content Planner
## Final Answer: 
**Comprehensive Content Plan Document**

**I. Introduction**
-----------------

* Article Title: "The Future of Artificial Intelligence: Trends, Key Players, and Noteworthy News"
* Subtitle: Exploring the latest developments in AI and their impact on society

**II. Target Audience Analysis**
------------------------------

* **Demographics:**
	+ Age: 25-45
	+ Education: College-educated professionals and enthusiasts
	+ Occupation: Tech-savvy individuals, researchers, and students
* **Interests:**
	+ Emerging technologies (AI, IoT, blockchain)
	+ Science and innovation
	+ Business and entrepreneurship
* **Pain Points:**
	+ Limited understanding of AI concepts and applications
	+ Difficulty in keeping up with the latest trends and advancements

**III. Detailed Content Outline**
---------------------------------

### A. Introduction to Artificial Intelligence

* Definition and brief history of AI
* Current state and prospects of AI development


### Displaying the Pydantic Results

Now that we have obtained our results using Pydantic output structure, let's move on to visualizing them. Instead of displaying the complete output as we did previously (`Markdown(result.raw)`), which is still an option, we can leverage the Pydantic task output structure to obtain the results in a more organized format.

We'll first print the **title**. Then, display the **summary**. Finally, we'll iterate over the different **paragraphs** and print them one by one.

You can also open the `Article.json` file that was created, which should have the same content as the result.

In [13]:
Markdown(result.pydantic.title)

The Future of Artificial Intelligence: Trends, Key Players, and Noteworthy News

In [14]:
Markdown(result.pydantic.summary)

As we navigate the complexities of the 21st century, Artificial Intelligence (AI) has become an integral part of our modern world. This article explores the latest developments in AI, its key trends and applications, industry leaders and startups, expert insights, and a call to action for readers to learn more about AI.

In [15]:
for idx, paragraph in enumerate(result.pydantic.paragraphs):
    print(f'Paragraph {idx}\n')
    display(Markdown(paragraph))

Paragraph 0



Artificial Intelligence refers to the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. The history of AI dates back to the 1950s when computer scientists began exploring ways to create machines that could simulate human thought processes.

Paragraph 1



Today, AI is a rapidly evolving field with its current state and prospects being shaped by breakthroughs in machine learning, deep learning, and natural language processing. As we explore these developments, it's essential to understand the key trends and applications of AI and how they are transforming industries and societies worldwide.

Paragraph 2



...

The next decade will be all about scaling up AI solutions to real-world problems. We need to make AI more accessible, more robust, and more explainable.

Paragraph 11



Dr. Andrew Ng, another prominent figure in the AI community, emphasized the importance of human-AI collaboration:

Paragraph 12



As we move forward with AI, it's crucial that we prioritize human values such as fairness, transparency, and accountability. Humans and machines should work together to create a better future for all.

## Useful resources

- [CrewAI Docs](https://docs.crewai.com/)
- [CrewAI Repository](https://github.com/crewAIInc/crewAI)
- [Langchain Docs](https://python.langchain.com/v0.2/docs/introduction/)